# Email Classification

In [ ]:
import pandas as pd
df = pd.read_csv('./data/email_spam.csv')
df.info()

In [ ]:
df.sample(5)

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby('Kategori').count()

## Preprocessing

### Membersihkan data dari yang bukan teks (angka, tanda baca)

In [ ]:
import re
import string

clean_text = lambda text: re.sub(f'[{string.punctuation}0-9]', '', text)
df['Pesan'] = df['Pesan'].apply(clean_text)

df.sample(5)

### Mengubah text menjadi lowercase

In [ ]:
df['Pesan'] = df['Pesan'].apply(lambda text: text.lower())
df.sample(5)

### Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# df['Pesan'] = df['Pesan'].apply(stemmer.stem)
# df.sample(5)

stemmer.stem('Kedewasaan')

In [ ]:
df = pd.read_csv('data/email_stemming.csv')
df.sample(5)

### Tokenisasi

In [ ]:
from nltk.tokenize import word_tokenize

df['Pesan'] = df['Pesan'].apply(word_tokenize)
df.sample(5)

### Remove Stopwords

In [ ]:
# from nltk.corpus import stopwords

# remove_stopwords = lambda words: [word for word in words if word not in stopwords.words('indonesian')]

# df['Pesan'] = df['Pesan'].apply(remove_stopwords)
# df.sample(5)

In [ ]:
df = pd.read_csv('data/email_stopword.csv')
df.sample(5)

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Pesan'])

# Menampilkan fitur yang dihasilkan dari TF-IDF 
# feature_names = vectorizer.get_feature_names_out()

# Mengonversi hasil TF-IDF ke DataFrame agar mudah dibaca
# tfidf_df = pd.DataFrame(X.toarray(), columns=feature_names)
vectorizer.get_feature_names_out()


## Modeling

In [ ]:
from sklearn.model_selection import train_test_split

y = df['Kategori']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.2f}")

print(classification_report(y_test, y_pred))